<h1>Creation of phenopackets from tabular data (individuals in columns)</h1>
<p>We will process <a href="https://pubmed.ncbi.nlm.nih.gov/29907796/" target="__blank">Diets, et al. (2019) SMARCB1 causes severe intellectual disability and choroid plexus hyperplasia with resultant hydrocephalus</a></p>

In [1]:
import phenopackets as php
from google.protobuf.json_format import MessageToDict, MessageToJson
from google.protobuf.json_format import Parse, ParseDict
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
import os
import sys
import numpy as np
import pyphetools
from pyphetools.creation import *
from pyphetools.visualization import *
print(f"pyphetools version {pyphetools.__version__}")

pyphetools version 0.6.4


<h2>Importing HPO data</h2>
<p>pyphetools uses the Human Phenotype Ontology (HPO) to encode phenotypic features. The recommended way of doing this is to ingest the hp.json file using HpoParser, which in turn creates an HpoConceptRecognizer object. </p>
<p>The HpoParser can accept a hpo_json_file argument if you want to use a specific file. If the argument is not passed, it will download the latext hp.json file from the HPO GitHub site and store it in a new subdirectory called hpo_data. It will not download the file if the file is already downloaded.</p>

In [2]:
parser = HpoParser()
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
PMID = "PMID:29907796"
title = "A recurrent de novo missense pathogenic variant in SMARCB1 causes severe intellectual disability and choroid plexus hyperplasia with resultant hydrocephalus"
metadata = MetaData(created_by="ORCID:0000-0002-5648-2155", pmid=PMID, pubmed_title=title)
metadata.default_versions_with_hpo(version=hpo_version)

<h2>Importing the supplemental table</h2>
<p>The Table of the Iwasawa et al (2019) paper was copied into an Excel file that is included in the data subfolder</p>
<p>Here, we use the pandas library to import this file (note that the Python package called openpyxl must be installed to read Excel files with pandas, although the library does not need to be imported in this notebook). pyphetools expects a pandas DataFrame as input, and users can choose any input format available for pandas include CSV, TSV, and Excel, or can use any other method to transform their input data into a Pandas DataFrame before using pyphetools.</p>

In [3]:
df = pd.read_excel('input/PMID_29907796.xlsx')

In [4]:
df

,Unnamed: 0,Patient 1,Patient 2,Patient 3,Patient 4,Count features
0,Pathogenic variant,c.110G>A,c.110G>A,c.110G>A,c.110G>A,NaN
1,NaN,p.Arg37His,p.Arg37His,p.Arg37His,p.Arg37His,NaN
2,Inheritance,De novo,De novo,De novo,De novo,NaN
3,Age at examination,9.5 y,5 y 8 mo,12 y,17 mo,NaN
4,Development,NaN,NaN,NaN,NaN,NaN
5,Intellectual disability,Severe,Severe,Severe,Severe,4/4 (100%)
6,Speech delay,Severe,Severe,Severe,Severe,4/4 (100%)
7,Motor delay,Severe,Severe,Severe,Severe,4/4 (100%)
8,Congenital anomalies,NaN,NaN,NaN,NaN,NaN
9,Congenital heart defect,-,+,-,+,2/4 (50%)


<h1>Converting to row-based format</h1>

In [5]:
dft = df.transpose()
dft.columns = dft.iloc[0]
dft.drop(dft.index[0], inplace=True)
dft = dft[dft.index.astype(str).str.contains('Patient')]
dft['patient_id'] = dft.index
dft.head()

Unnamed: 0,Pathogenic variant,NaN,Inheritance,Age at examination,Development,Intellectual disability,Speech delay,Motor delay,Congenital anomalies,Congenital heart defect,...,Brachycephaly,Joint hypermobility,Hip dysplasia,Contractures,Other,Obstructive sleep apnea,Precocious puberty,(History of) anemia,Thrombocytopenia,patient_id
Patient 1,c.110G>A,p.Arg37His,De novo,9.5 y,NaN,Severe,Severe,Severe,NaN,-,...,+,-,-,+,NaN,+,-,+,-,Patient 1
Patient 2,c.110G>A,p.Arg37His,De novo,5 y 8 mo,NaN,Severe,Severe,Severe,NaN,+,...,-,+,-,-,NaN,+,+,+,-,Patient 2
Patient 3,c.110G>A,p.Arg37His,De novo,12 y,NaN,Severe,Severe,Severe,NaN,-,...,-,+,-,+,NaN,+,-,+,-,Patient 3
Patient 4,c.110G>A,p.Arg37His,De novo,17 mo,NaN,Severe,Severe,Severe,NaN,+,...,+,+,+,-,NaN,+,NaN,-,+,Patient 4


Some column names might include spaces in front or after, and a couple of columns are subheadings and only contain NaNs, so lets correct that:

In [6]:
dft.columns = dft.columns.str.strip()
dft = dft.dropna(axis=1, how='all')
dft['Intellectual disability, severe'] = dft['Intellectual disability']
del dft['Intellectual disability']
dft.head()

Unnamed: 0,Pathogenic variant,NaN,Inheritance,Age at examination,Speech delay,Motor delay,Congenital heart defect,Laryngomalacia,Kidney anomalies,Genital anomalies,...,Brachycephaly,Joint hypermobility,Hip dysplasia,Contractures,Obstructive sleep apnea,Precocious puberty,(History of) anemia,Thrombocytopenia,patient_id,"Intellectual disability, severe"
Patient 1,c.110G>A,p.Arg37His,De novo,9.5 y,Severe,Severe,-,-,-,+,...,+,-,-,+,+,-,+,-,Patient 1,Severe
Patient 2,c.110G>A,p.Arg37His,De novo,5 y 8 mo,Severe,Severe,+,-,+,-,...,-,+,-,-,+,+,+,-,Patient 2,Severe
Patient 3,c.110G>A,p.Arg37His,De novo,12 y,Severe,Severe,-,-,+,+,...,-,+,-,+,+,-,+,-,Patient 3,Severe
Patient 4,c.110G>A,p.Arg37His,De novo,17 mo,Severe,Severe,+,+,+,-,...,+,+,+,-,+,NaN,-,+,Patient 4,Severe


<h2>Column mappers</h2>
<p>Please see the notebook "Create phenopackets from tabular data with individuals in rows" for explanations. In the following cell we create a dictionary for the ColumnMappers. Note that the code is identical except that we use the df.loc function to get the corresponding row data</p>

In [7]:
hpo_cr = parser.get_hpo_concept_recognizer()
generator = SimpleColumnMapperGenerator(df=dft.loc[:,['Intellectual disability, severe','Speech delay', 'Motor delay']],
                                                    observed='Severe',
                                                    excluded='-',
                                                    hpo_cr=hpo_cr)

column_mapper_d = generator.try_mapping_columns()
print(generator.get_mapped_columns())

['Intellectual disability, severe', 'Speech delay', 'Motor delay']


In [8]:
print(generator.get_unmapped_columns())

[]


In [9]:
generator_plus = SimpleColumnMapperGenerator(df=dft.loc[:,'Congenital heart defect': 'patient_id'],
                                                    observed='+',
                                                    excluded='-',
                                                    hpo_cr=hpo_cr)

column_mapper_d_plus = generator_plus.try_mapping_columns()

In [10]:
col_not_found_plus = generator_plus.get_unmapped_columns()

Lets merge the two dicts and the lists, to see which columns are not mapped yet.

In [11]:
column_mapper_d.update(column_mapper_d_plus)
col_not_found = generator.get_unmapped_columns()
col_not_found.extend(col_not_found_plus)
col_not_found = list(set(col_not_found))
print(col_not_found)

['Kidney anomalies', 'CVI', 'Other eye problems', 'Childhood hypotonia', 'Eye movement disorder', 'patient_id', '(History of) anemia', 'Contractures']


In [12]:
correct_hpo_ids = {}
correct_hpo_ids['Speech delay'] = 'HP:0000750'
correct_hpo_ids['Eye movement disorder'] = 'HP:0000496'
correct_hpo_ids['Contractures'] = 'HP:0001371'
correct_hpo_ids['Kidney anomalies'] = 'HP:0000077'
correct_hpo_ids['CVI'] = 'HP:0100704'
correct_hpo_ids['Genital anomalies'] = 'HP:0000078'
correct_hpo_ids['Congenital heart defect'] = 'HP:0001627'
correct_hpo_ids['Hearing loss'] = 'HP:0000365'
correct_hpo_ids['Other eye problems'] = 'HP:0000478'
for k,v in correct_hpo_ids.items():
    if v == 'HP:0000750':
        t_mapper = SimpleColumnMapper(hpo_id=v,
                                      hpo_label=hpo_cr._id_to_primary_label[v],
                                      observed='Severe',
                                      excluded='-')
    else:
        t_mapper = SimpleColumnMapper(hpo_id=v,
                                      hpo_label=hpo_cr._id_to_primary_label[v],
                                      observed='+',
                                      excluded='-')
    print(t_mapper.preview_column(dft[k]))
    column_mapper_d[k] = t_mapper

                                                   term    status
0  Delayed speech and language development (HP:0000750)  observed
1  Delayed speech and language development (HP:0000750)  observed
2  Delayed speech and language development (HP:0000750)  observed
3  Delayed speech and language development (HP:0000750)  observed
                                       term    status
0  Abnormality of eye movement (HP:0000496)  excluded
1  Abnormality of eye movement (HP:0000496)  observed
2  Abnormality of eye movement (HP:0000496)  observed
3  Abnormality of eye movement (HP:0000496)  observed
                               term    status
0  Flexion contracture (HP:0001371)  observed
1  Flexion contracture (HP:0001371)  excluded
2  Flexion contracture (HP:0001371)  observed
3  Flexion contracture (HP:0001371)  excluded
                                     term    status
0  Abnormality of the kidney (HP:0000077)  excluded
1  Abnormality of the kidney (HP:0000077)  observed
2  Abnormality

<h2>Variant Data</h2>
<p>The variant data (HGVS< transcript) is listed in the Variant (hg19, NM_015133.4) column.</p>

In [17]:
genome = 'hg38'
default_genotype = 'heterozygous'
SMARCB1_transcript='NM_003073.3'
variant_list = dft['Pathogenic variant'].unique()
print(variant_list)
vvalidator = VariantValidator(genome_build=genome, transcript=SMARCB1_transcript)
# There is just one variant in this dataset
SMARCB1_var = vvalidator.encode_hgvs('c.110G>A')
print(SMARCB1_var)
variant_d = {'c.110G>A' : SMARCB1_var}


['c.110G>A']
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_003073.3%3Ac.110G>A/NM_003073.3?content-type=application%2Fjson
chr22:23791772G>A


In [18]:
varMapper = VariantColumnMapper(variant_d=variant_d, variant_column_name='Pathogenic variant', default_genotype=default_genotype)

<h1>Demographic data</h1>
<p>pyphetools can be used to capture information about age, sex, and individual identifiers. This information is stored in a map of "IndividualMapper" objects. Special treatment may be required for the indifiers, which may be used as the column names or row index.</p>

In [19]:
#age is in years and months, so manually correct it
dft['Age at examination'] = np.array([10, 6, 12, 1], dtype=int)
ageMapper = AgeColumnMapper.by_year('Age at examination')
ageMapper.preview_column(dft['Age at examination'])

,original column contents,age
0,10,P10Y
1,6,P6Y
2,12,P12Y
3,1,P1Y


In [20]:
#sex is not in columns, since it were all females in this paper
dft['Sex'] = ['Female'] * 4
sexMapper = SexColumnMapper(male_symbol='Male', female_symbol='Female', column_name='Sex')
sexMapper.preview_column(dft['Sex'])

,original column contents,sex
0,Female,FEMALE
1,Female,FEMALE
2,Female,FEMALE
3,Female,FEMALE


In [22]:
pmid = "PMID:29907796"
encoder = CohortEncoder(df=dft, 
                        hpo_cr=hpo_cr, 
                        column_mapper_d=column_mapper_d, 
                        individual_column_name="patient_id", 
                        agemapper=ageMapper, 
                        sexmapper=sexMapper,
                        variant_mapper=varMapper, 
                        metadata=metadata,
                        pmid=PMID)
encoder.set_disease(disease_id='CUSTOM:Diets2019', label='Diets_2019_SMARCB1')

In [23]:
individuals = encoder.get_individuals()

In [16]:
i1 = individuals[0]
phenopacket1 = i1.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh())
json_string = MessageToJson(phenopacket1)
print(json_string)

{
  "id": "Patient 1",
  "subject": {
    "id": "Patient 1",
    "timeAtLastEncounter": {
      "age": {
        "iso8601duration": "P10Y"
      }
    },
    "sex": "FEMALE"
  },
  "phenotypicFeatures": [
    {
      "type": {
        "id": "HP:0010864",
        "label": "Intellectual disability, severe"
      },
      "onset": {
        "age": {
          "iso8601duration": "P10Y"
        }
      }
    },
    {
      "type": {
        "id": "HP:0000750",
        "label": "Delayed speech and language development"
      },
      "onset": {
        "age": {
          "iso8601duration": "P10Y"
        }
      }
    },
    {
      "type": {
        "id": "HP:0001270",
        "label": "Motor delay"
      },
      "onset": {
        "age": {
          "iso8601duration": "P10Y"
        }
      }
    },
    {
      "type": {
        "id": "HP:0001627",
        "label": "Abnormal heart morphology"
      },
      "excluded": true,
      "onset": {
        "age": {
          "iso8601duration": "

In [24]:
from IPython.display import HTML, display
phenopackets = [i.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh()) for i in individuals]
table = PhenopacketTable(phenopacket_list=phenopackets)
display(HTML(table.to_html()))

In [17]:
output_directory = "../../phenopackets/SMARCB1/"
encoder.output_phenopackets(outdir=output_directory)

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_003073.3%3Ac.110G>A/NM_003073.3
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_003073.3%3Ac.110G>A/NM_003073.3
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_003073.3%3Ac.110G>A/NM_003073.3
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_003073.3%3Ac.110G>A/NM_003073.3
Wrote 4 phenopackets to ../../phenopackets/SMARCB1/
